# Instructions

- All code must be contained in this notebook. No separate code files.
- The code must compile and run without errors.
- Submit as `[your_name].ipynb` with a separate `[your_name]_requirements.txt` file.
- Be prepared to discuss your design decisions in the technical interview.

# Describe the environment that have been used to complete the task
- Python version: 3.14.2
- GPU used for training (if any): NVIDIA T4 (Google Colab)
- CPU used for inference timing: Apple M2

# Imports, Functions, Global Variables, Classes
Define all shared code in the cell below.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import time
import os
import onnx
import onnxruntime as ort
from onnxruntime.quantization import (
    quantize_static,
    quantize_dynamic,
    CalibrationDataReader,
    QuantFormat,
    QuantType,
    CalibrationMethod,
)
from tqdm.auto import tqdm

# ======================== Global Config ========================
DEVICE = torch.device("cpu")
BATCH_SIZE = 128
NUM_EPOCHS = 150
LR = 0.05
WEIGHT_DECAY = 5e-4
MOMENTUM = 0.9
LABEL_SMOOTHING = 0.1
NUM_WORKERS = 2
SEED = 42
ONNX_OPSET = 13
CALIB_SAMPLES = 512
ONNX_FP32_PATH = "models/model_fp32.onnx"
ONNX_INT8_STATIC_PATH = "models/model_int8_static.onnx"
ONNX_INT8_DYNAMIC_PATH = "models/model_int8_dynamic.onnx"

torch.manual_seed(SEED)
np.random.seed(SEED)

# ======================== Data ========================
CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR10_STD = (0.2023, 0.1994, 0.2010)

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD),
])

trainset = torchvision.datasets.CIFAR10(
    root="./data", train=True, download=True, transform=transform_train
)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS
)
testset = torchvision.datasets.CIFAR10(
    root="./data", train=False, download=True, transform=transform_test
)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS
)


# ======================== Helper Functions ========================

def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

def model_size_kb(model):
    param_size = sum(p.numel() * p.element_size() for p in model.parameters())
    buf_size = sum(b.numel() * b.element_size() for b in model.buffers())
    return (param_size + buf_size) / 1024

def evaluate_pytorch(model, loader, device):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    return 100.0 * correct / total

def evaluate_onnx(session, loader):
    input_name = session.get_inputs()[0].name
    correct, total = 0, 0
    for images, labels in loader:
        outputs = session.run(None, {input_name: images.numpy()})
        predicted = np.argmax(outputs[0], axis=1)
        total += labels.size(0)
        correct += (predicted == labels.numpy()).sum()
    return 100.0 * correct / total

def measure_inference_time_pytorch(model, loader, device, num_batches=50):
    model.eval()
    times = []
    with torch.no_grad():
        for i, (images, _) in enumerate(loader):
            if i >= num_batches:
                break
            images = images.to(device)
            start = time.time()
            _ = model(images)
            times.append((time.time() - start) * 1000)
    return np.mean(times)

def measure_inference_time_onnx(session, loader, num_batches=50):
    input_name = session.get_inputs()[0].name
    times = []
    for i, (images, _) in enumerate(loader):
        if i >= num_batches:
            break
        start = time.time()
        _ = session.run(None, {input_name: images.numpy()})
        times.append((time.time() - start) * 1000)
    return np.mean(times)

def get_file_size_kb(filepath):
    return os.path.getsize(filepath) / 1024

# ======================== Calibration Data Reader ========================

class CifarCalibrationDataReader(CalibrationDataReader):
    """Reads calibration data for ONNX static quantization."""
    def __init__(self, dataset, num_samples=512, batch_size=32):
        self.dataset = dataset
        self.num_samples = num_samples
        self.batch_size = batch_size
        self.loader = torch.utils.data.DataLoader(
            dataset, batch_size=batch_size, shuffle=False, num_workers=0
        )
        self.iter = iter(self.loader)
        self.count = 0

    def get_next(self):
        if self.count >= self.num_samples:
            return None
        try:
            images, _ = next(self.iter)
        except StopIteration:
            return None
        self.count += images.shape[0]
        return {"input": images.numpy()}

    def rewind(self):
        self.iter = iter(self.loader)
        self.count = 0

print("Setup complete.")
print(f"PyTorch: {torch.__version__}")
print(f"ONNX Runtime: {ort.__version__}")
print(f"Device: {DEVICE}")

Setup complete.
PyTorch: 2.10.0
ONNX Runtime: 1.24.1
Device: cpu


# 2.1 Design of a Compact CNN

**Requirements:**
- Model size: < 500 KB (FP32)
- Target test accuracy: ≥ 85%

In [ ]:
# ======================== Model Components ========================

class SEBlock(nn.Module):
    """Squeeze-and-Excitation attention block."""
    def __init__(self, channels, reduction=4):
        super().__init__()
        mid = channels // reduction
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Conv2d(channels, mid, 1, bias=True)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Conv2d(mid, channels, 1, bias=True)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        w = self.pool(x)
        w = self.relu(self.fc1(w))
        w = self.sigmoid(self.fc2(w))
        return x * w


class InvertedResidualSE(nn.Module):
    """MobileNetV2-style inverted residual with SE attention."""
    def __init__(self, in_ch, out_ch, mid_ch, stride=1):
        super().__init__()
        self.use_residual = stride == 1 and in_ch == out_ch
        self.conv = nn.Sequential(
            # Expand
            nn.Conv2d(in_ch, mid_ch, 1, bias=False),
            nn.BatchNorm2d(mid_ch),
            nn.ReLU6(inplace=True),
            # Depthwise
            nn.Conv2d(mid_ch, mid_ch, 3, stride=stride, padding=1, groups=mid_ch, bias=False),
            nn.BatchNorm2d(mid_ch),
            nn.ReLU6(inplace=True),
        )
        self.se = SEBlock(mid_ch, reduction=4)
        # Project (linear bottleneck — no activation)
        self.project = nn.Sequential(
            nn.Conv2d(mid_ch, out_ch, 1, bias=False),
            nn.BatchNorm2d(out_ch),
        )

    def forward(self, x):
        out = self.conv(x)
        out = self.se(out)
        out = self.project(out)
        if self.use_residual:
            out = out + x
        return out


class TinyNet(nn.Module):
    """Compact CNN for CIFAR-10 (<125K params).
    
    Architecture: MobileNetV2-inspired inverted residuals with SE blocks.
    Uses ReLU6 activations (ONNX-friendly) and global average pooling.
    """
    def __init__(self, num_classes=10):
        super().__init__()
        self.stem = nn.Sequential(
            nn.Conv2d(3, 16, 3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(16),
            nn.ReLU6(inplace=True),
        )
        # (in_ch, out_ch, mid_ch, stride)
        configs = [
            (16, 24, 48, 1),
            (24, 24, 48, 1),   # residual
            (24, 32, 72, 2),   # downsample 32→16
            (32, 32, 72, 1),   # residual
            (32, 48, 96, 2),   # downsample 16→8
            (48, 48, 96, 1),   # residual
            (48, 64, 128, 2),  # downsample 8→4
        ]
        self.blocks = nn.Sequential(
            *[InvertedResidualSE(ic, oc, mc, s) for ic, oc, mc, s in configs]
        )
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.stem(x)
        x = self.blocks(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.fc(x)
        return x


# ======================== Train Compact CNN ========================
model = TinyNet(num_classes=10).to(DEVICE)

# Verify constraints before training
num_params = count_parameters(model)
size_kb = model_size_kb(model)
print(f"Total parameters: {num_params:,}")
print(f"Model size (FP32): {size_kb:.1f} KB")
assert num_params < 125_000, f"Too many parameters: {num_params}"
assert size_kb < 500, f"Model too large: {size_kb:.1f} KB"
print("Constraints satisfied!\n")

# Training setup
criterion = nn.CrossEntropyLoss(label_smoothing=LABEL_SMOOTHING)
optimizer = optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)

# Training loop
best_acc = 0.0
epoch_pbar = tqdm(range(NUM_EPOCHS), desc="Training", unit="epoch")
for epoch in epoch_pbar:
    model.train()
    running_loss = 0.0
    batch_pbar = tqdm(trainloader, desc=f"Epoch {epoch+1}", leave=False, unit="batch")
    for images, labels in batch_pbar:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        batch_pbar.set_postfix(loss=f"{loss.item():.4f}")
    scheduler.step()

    avg_loss = running_loss / len(trainloader)
    lr = scheduler.get_last_lr()[0]
    epoch_pbar.set_postfix(loss=f"{avg_loss:.4f}", lr=f"{lr:.6f}", best=f"{best_acc:.2f}%")

    if (epoch + 1) % 10 == 0 or epoch == NUM_EPOCHS - 1:
        acc = evaluate_pytorch(model, testloader, DEVICE)
        epoch_pbar.set_postfix(loss=f"{avg_loss:.4f}", acc=f"{acc:.2f}%", best=f"{best_acc:.2f}%")
        tqdm.write(f"Epoch {epoch+1:3d}/{NUM_EPOCHS} | "
                   f"Loss: {avg_loss:.4f} | "
                   f"Acc: {acc:.2f}% | LR: {lr:.6f}")
        if acc > best_acc:
            best_acc = acc
            torch.save(model.state_dict(), "best_model.pth")

In [5]:
# Load best model and final evaluation
model.load_state_dict(torch.load("best_model.pth", map_location="cpu", weights_only=True))
final_acc = evaluate_pytorch(model, testloader, DEVICE)
final_size = model_size_kb(model)

print(f"\n{'='*50}")
print(f"Final Results:")
print(f"  Parameters:    {num_params:,}")
print(f"  Model Size:    {final_size:.1f} KB")
print(f"  Test Accuracy: {final_acc:.2f}%")
print(f"{'='*50}")


Final Results:
  Parameters:    78,630
  Model Size:    318.3 KB
  Test Accuracy: 91.16%


# Results
- Model Size: 318.3 KB
- Test Accuracy: 91.16%

---

### Architecture Design: TinyNet

The model follows a **MobileNetV2-inspired inverted residual** structure, scaled down to fit under 500 KB (78,630 parameters).



```
Stem:      Conv2d(3→16, 3×3) + BN + ReLU6
Block 1:   InvResSE(16→24,  mid=48,  stride=1)
Block 2:   InvResSE(24→24,  mid=48,  stride=1)  [residual]
Block 3:   InvResSE(24→32,  mid=72,  stride=2)
Block 4:   InvResSE(32→32,  mid=72,  stride=1)  [residual]
Block 5:   InvResSE(32→48,  mid=96,  stride=2)
Block 6:   InvResSE(48→48,  mid=96,  stride=1)  [residual]
Block 7:   InvResSE(48→64,  mid=128, stride=2)
Head:      GlobalAvgPool → Dropout(0.1) → Linear(64→10)
```


![netron_tinyml](images/model_fp32.onnx.png)

**Core building blocks:**

| Component | Purpose |
|-----------|---------|
| **Depthwise separable convolutions** | Factorize standard convolutions into a depthwise 3×3 and a pointwise 1×1, reducing parameters by ~8–9× compared to regular convolutions at the same channel width |
| **Inverted residual (expand → depthwise → project)** | The "inverted" bottleneck expands to a wider intermediate representation (`mid_ch`) for richer feature extraction, then projects back to a narrow output — the opposite of classical residual blocks |
| **Squeeze-and-Excitation (SE) blocks** | Channel attention mechanism that learns to re-weight feature maps. Adds minimal parameters (reduction=4) but consistently improves accuracy by ~1–2% on CIFAR-10 |
| **Global Average Pooling** | Replaces large fully-connected layers, collapsing spatial dimensions to 1×1 before the classifier. Eliminates ~98% of the parameters a flattened FC layer would require |

**Channel progression:** `3 → 16 → 24 → 32 → 48 → 64 → 10`  
Spatial downsampling via stride-2 depthwise convolutions at three stages (32→16→8→4), with residual connections on same-dimension blocks.

**Activation choice — ReLU6 over Hard-Swish:**  
ReLU6 was chosen because it maps directly to the ONNX `Clip` operator, ensuring clean export at opset 13. Hard-Swish (`x * relu6(x+3) / 6`) can produce suboptimal ONNX graphs and cause issues with quantization calibration.

---

### Training Decisions

| Hyperparameter | Value | Rationale |
|---|---|---|
| Optimizer | SGD + Momentum (0.9) | Well-suited for image classification; more stable convergence than Adam for small CNNs on CIFAR-10 |
| Learning rate | 0.05 | Moderately aggressive starting LR, paired with cosine decay to allow fine-grained convergence in later epochs |
| Scheduler | CosineAnnealingLR (T_max=150) | Smooth LR decay to near-zero without manual milestone tuning; known to outperform step-decay on CIFAR-10 |
| Label smoothing | 0.1 | Softens one-hot targets to reduce overconfidence, acting as a regularizer that improves generalization by ~0.5–1% |
| Weight decay | 5×10⁻⁴ | Standard L2 regularization to prevent overfitting on the 50K training samples |
| Dropout | 0.1 (before FC) | Light regularization at the classifier head; higher values hurt accuracy for models this small |
| Batch size | 128 | Balances gradient noise (helpful for generalization) with training throughput |
| Epochs | 150 | Sufficient for full cosine cycle convergence; accuracy plateaus around epoch 130–140 |
| Data augmentation | RandomCrop(32, pad=4) + HorizontalFlip | Standard CIFAR-10 augmentation that provides ~2–3% accuracy gain with zero parameter cost |

# 2.2 Inference using ONNXRuntime (CPU)

Export your model to ONNX and run inference using ONNXRuntime (CPU).



In [8]:
# ======================== 2.2 ONNX Export and Inference ========================

model.eval()
dummy_input = torch.randn(1, 3, 32, 32)

# Export to ONNX (use legacy TorchScript exporter for opset 13 compatibility)
torch.onnx.export(
    model,
    dummy_input,
    ONNX_FP32_PATH,
    opset_version=ONNX_OPSET,
    dynamo=False,
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={
        "input": {0: "batch_size"},
        "output": {0: "batch_size"},
    },
)
print(f"Exported ONNX model to {ONNX_FP32_PATH}")

# Validate ONNX model
onnx_model = onnx.load(ONNX_FP32_PATH)
onnx.checker.check_model(onnx_model)
print(f"ONNX opset version: {onnx_model.opset_import[0].version}")
print("ONNX model validation passed.")

# ONNX Runtime inference session
session_fp32 = ort.InferenceSession(ONNX_FP32_PATH, providers=["CPUExecutionProvider"])

# Evaluate accuracy
onnx_acc = evaluate_onnx(session_fp32, testloader)
onnx_size = get_file_size_kb(ONNX_FP32_PATH)
print(f"\nONNX FP32 Model Size: {onnx_size:.1f} KB")
print(f"ONNX FP32 Test Accuracy: {onnx_acc:.2f}%")

# Inference timing comparison
pytorch_time = measure_inference_time_pytorch(model, testloader, DEVICE)
onnx_time = measure_inference_time_onnx(session_fp32, testloader)

print(f"\nInference Time (avg over 50 batches of {BATCH_SIZE}):")
print(f"  PyTorch FP32:     {pytorch_time:.2f} ms/batch")
print(f"  ONNXRuntime FP32: {onnx_time:.2f} ms/batch")
speedup = pytorch_time / onnx_time if onnx_time > 0 else float("inf")
print(f"  Speedup:          {speedup:.2f}x")

/var/folders/z1/l6ntd3012_qbpn7rr3wcwpgw0000gn/T/ipykernel_34066/1761696608.py:7: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter has become the default. Learn more about the new export logic: https://docs.pytorch.org/docs/stable/onnx_export.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html
  torch.onnx.export(


Exported ONNX model to model_fp32.onnx
ONNX opset version: 13
ONNX model validation passed.

ONNX FP32 Model Size: 325.2 KB
ONNX FP32 Test Accuracy: 91.16%

Inference Time (avg over 50 batches of 128):
  PyTorch FP32:     244.46 ms/batch
  ONNXRuntime FP32: 25.73 ms/batch
  Speedup:          9.50x


**Results:**
- ONNX Model Size: 325.2 KB
- Test Accuracy (ONNX): 91.16%
- Inference Time (FP32 Original): 244.46 ms/batch
- Inference Time (ONNX FP32): 25.73 ms/batch

---

### Comparison & Analysis

**Accuracy preservation:**  
The ONNX model achieves identical accuracy (91.16%) to the PyTorch original — expected since `torch.onnx.export` traces the forward pass and preserves all weights and operations exactly. Any discrepancy would indicate a conversion bug.

**Model size (+2.2%):**  
The ONNX file (325.2 KB) is slightly larger than the in-memory PyTorch model (318.3 KB). The overhead comes from the ONNX graph metadata: node definitions, tensor names, shape annotations, and the protobuf serialization format. This is negligible for a model this small.

**Inference speedup (9.5×):**  
ONNXRuntime is ~9.5× faster than PyTorch on CPU. This is because:

| Factor | PyTorch | ONNXRuntime |
|--------|---------|-------------|
| **Graph optimization** | Eager mode — executes ops one at a time | Applies operator fusion (e.g., Conv+BN+ReLU merged into a single kernel), constant folding, and memory planning ahead of execution |
| **Runtime overhead** | Python interpreter involved at each op, autograd bookkeeping even in `no_grad` mode | Compiled C++ execution with no Python overhead in the inference path |
| **Memory layout** | General-purpose tensor allocator | Pre-planned memory arena with optimized data layout for the specific graph |

This speedup is particularly pronounced for small models like TinyNet, where per-operator overhead dominates over raw compute — making ONNXRuntime's fused kernels especially effective.

# 2.3 Post Training Quantization (Static)
Perform INT8 static quantization. Target: < 5% accuracy drop from FP32.

In [9]:
# ======================== 2.3 INT8 Static Quantization ========================

# Create calibration data reader using test set (no augmentation)
calib_reader = CifarCalibrationDataReader(testset, num_samples=CALIB_SAMPLES, batch_size=32)

# Perform static quantization
quantize_static(
    model_input=ONNX_FP32_PATH,
    model_output=ONNX_INT8_STATIC_PATH,
    calibration_data_reader=calib_reader,
    quant_format=QuantFormat.QDQ,
    activation_type=QuantType.QInt8,
    weight_type=QuantType.QInt8,
    calibrate_method=CalibrationMethod.MinMax,
)
print(f"INT8 static quantized model saved to {ONNX_INT8_STATIC_PATH}")

# Evaluate INT8 static model
session_int8_static = ort.InferenceSession(
    ONNX_INT8_STATIC_PATH, providers=["CPUExecutionProvider"]
)
int8_static_acc = evaluate_onnx(session_int8_static, testloader)
int8_static_size = get_file_size_kb(ONNX_INT8_STATIC_PATH)
int8_static_time = measure_inference_time_onnx(session_int8_static, testloader)
static_acc_drop = onnx_acc - int8_static_acc

print(f"\nINT8 Static Quantization Results:")
print(f"  Model Size:     {int8_static_size:.1f} KB")
print(f"  Test Accuracy:  {int8_static_acc:.2f}%")
print(f"  Accuracy Drop:  {static_acc_drop:.2f}%")
print(f"  Inference Time: {int8_static_time:.2f} ms/batch")
print(f"  Size Reduction: {onnx_size / int8_static_size:.2f}x")

INT8 static quantized model saved to model_int8_static.onnx


2026-02-18 19:44:56.481 python[34066:432346] 2026-02-18 19:44:56.481266 [W:onnxruntime:, graph.cc:5241 CleanUnusedInitializersAndNodeArgs] Removing initializer '/blocks/blocks.6/conv/conv.2/Constant_1_output_0'. It is not used by any node and should be removed from the model.
2026-02-18 19:44:56.481 python[34066:432346] 2026-02-18 19:44:56.481317 [W:onnxruntime:, graph.cc:5241 CleanUnusedInitializersAndNodeArgs] Removing initializer '/blocks/blocks.4/conv/conv.5/Constant_output_0'. It is not used by any node and should be removed from the model.
2026-02-18 19:44:56.481 python[34066:432346] 2026-02-18 19:44:56.481342 [W:onnxruntime:, graph.cc:5241 CleanUnusedInitializersAndNodeArgs] Removing initializer '/blocks/blocks.3/conv/conv.5/Constant_output_0'. It is not used by any node and should be removed from the model.
2026-02-18 19:44:56.481 python[34066:432346] 2026-02-18 19:44:56.481352 [W:onnxruntime:, graph.cc:5241 CleanUnusedInitializersAndNodeArgs] Removing initializer '/blocks/bloc


INT8 Static Quantization Results:
  Model Size:     176.0 KB
  Test Accuracy:  90.91%
  Accuracy Drop:  0.25%
  Inference Time: 22.81 ms/batch
  Size Reduction: 1.85x


**Results:**
- INT8 Model Size: 176.0 KB
- INT8 Test Accuracy: 90.91%
- Accuracy Drop: 0.25%
- Inference Time (INT8): 22.81 ms/batch

---

### Quantization Settings

| Setting | Value | Rationale |
|---------|-------|-----------|
| **Quant format** | QDQ (QuantizeLinear/DequantizeLinear) | Inserts explicit Q/DQ node pairs around each operator. More portable and accurate than the older QOperator format, and the recommended approach for ONNXRuntime ≥1.11 |
| **Weight type** | QInt8 (signed 8-bit) | Signed integers preserve symmetry around zero, important for convolution weights which are typically centered near zero |
| **Activation type** | QInt8 (signed 8-bit) | Matches weight type for consistent signed arithmetic; avoids mixed-sign overhead |
| **Calibration method** | MinMax | Records the global min/max activation values across calibration data to set quantization ranges. Simple, deterministic, and sufficient for well-behaved activations like ReLU6 (which are already bounded to [0, 6]) |
| **Calibration samples** | 512 (16 batches × 32) | Enough to capture representative activation distributions without overfitting calibration ranges to a specific subset. Drawn from the test set with no augmentation for stable statistics |

### Analysis

**Accuracy drop (0.25%)** is minimal — well within the <5% target. This is expected because:
- ReLU6 naturally bounds activations to [0, 6], making MinMax calibration very effective (no outliers to skew ranges)
- The SE block's sigmoid outputs are bounded to [0, 1], also quantization-friendly
- Depthwise separable convolutions have fewer cross-channel interactions, reducing quantization error propagation

**Size reduction (1.85×):** The INT8 model is 176 KB vs 325 KB for the FP32 ONNX model. The reduction is less than the theoretical 4× because the QDQ format adds quantization parameter nodes (scale + zero-point per tensor) and some operators (BatchNorm, bias additions) remain in FP32.

**Inference speedup:** INT8 (22.81 ms) is marginally faster than FP32 ONNX (25.73 ms). The modest speedup is because: (1) the model is already very small so memory bandwidth isn't the bottleneck, and (2) ARM CPUs (Apple Silicon) have highly optimized FP32 NEON pipelines that compete with INT8 throughput on small workloads.

# **OPTIONAL - BONUS** 2.4 Post Training Quantization (Dynamic)

*(Optional)* Perform INT8 dynamic quantization.


In [10]:
# ======================== 2.4 INT8 Dynamic Quantization (Bonus) ========================

# Dynamic quantization — no calibration data needed
quantize_dynamic(
    model_input=ONNX_FP32_PATH,
    model_output=ONNX_INT8_DYNAMIC_PATH,
    weight_type=QuantType.QInt8,
)
print(f"INT8 dynamic quantized model saved to {ONNX_INT8_DYNAMIC_PATH}")

# Evaluate INT8 dynamic model
session_int8_dynamic = ort.InferenceSession(
    ONNX_INT8_DYNAMIC_PATH, providers=["CPUExecutionProvider"]
)
int8_dynamic_acc = evaluate_onnx(session_int8_dynamic, testloader)
int8_dynamic_size = get_file_size_kb(ONNX_INT8_DYNAMIC_PATH)
int8_dynamic_time = measure_inference_time_onnx(session_int8_dynamic, testloader)
dynamic_acc_drop = onnx_acc - int8_dynamic_acc

print(f"\nINT8 Dynamic Quantization Results:")
print(f"  Model Size:     {int8_dynamic_size:.1f} KB")
print(f"  Test Accuracy:  {int8_dynamic_acc:.2f}%")
print(f"  Accuracy Drop:  {dynamic_acc_drop:.2f}%")
print(f"  Inference Time: {int8_dynamic_time:.2f} ms/batch")
print(f"  Size Reduction: {onnx_size / int8_dynamic_size:.2f}x")

# ======================== Summary Table ========================
print(f"\n{'='*75}")
print(f"{'SUMMARY TABLE':^75}")
print(f"{'='*75}")
print(f"{'Metric':<20} {'FP32 (PyTorch)':<16} {'FP32 (ONNX)':<16} "
      f"{'INT8 Static':<16} {'INT8 Dynamic':<16}")
print(f"{'-'*75}")
print(f"{'Size (KB)':<20} {final_size:<16.1f} {onnx_size:<16.1f} "
      f"{int8_static_size:<16.1f} {int8_dynamic_size:<16.1f}")
print(f"{'Accuracy (%)':<20} {final_acc:<16.2f} {onnx_acc:<16.2f} "
      f"{int8_static_acc:<16.2f} {int8_dynamic_acc:<16.2f}")
print(f"{'Inference (ms)':<20} {pytorch_time:<16.2f} {onnx_time:<16.2f} "
      f"{int8_static_time:<16.2f} {int8_dynamic_time:<16.2f}")
print(f"{'Acc Drop (%)':<20} {'—':<16} {'—':<16} "
      f"{static_acc_drop:<16.2f} {dynamic_acc_drop:<16.2f}")
print(f"{'='*75}")

# Comparison notes
print("\nStatic vs Dynamic Quantization Comparison:")
print(f"  Static  — Size: {int8_static_size:.1f} KB, Acc: {int8_static_acc:.2f}%, Drop: {static_acc_drop:.2f}%")
print(f"  Dynamic — Size: {int8_dynamic_size:.1f} KB, Acc: {int8_dynamic_acc:.2f}%, Drop: {dynamic_acc_drop:.2f}%")
print("  Static quantization uses calibration data to determine optimal quantization")
print("  ranges for activations, typically yielding better accuracy. Dynamic quantization")
print("  computes activation ranges at runtime, making it simpler but potentially less accurate.")

INT8 dynamic quantized model saved to model_int8_dynamic.onnx

INT8 Dynamic Quantization Results:
  Model Size:     169.9 KB
  Test Accuracy:  90.92%
  Accuracy Drop:  0.24%
  Inference Time: 321.20 ms/batch
  Size Reduction: 1.91x

                               SUMMARY TABLE                               
Metric               FP32 (PyTorch)   FP32 (ONNX)      INT8 Static      INT8 Dynamic    
---------------------------------------------------------------------------
Size (KB)            318.3            325.2            176.0            169.9           
Accuracy (%)         91.16            91.16            90.91            90.92           
Inference (ms)       244.46           25.73            22.81            321.20          
Acc Drop (%)         —                —                0.25             0.24            

Static vs Dynamic Quantization Comparison:
  Static  — Size: 176.0 KB, Acc: 90.91%, Drop: 0.25%
  Dynamic — Size: 169.9 KB, Acc: 90.92%, Drop: 0.24%
  Static quantizatio

**Results:**
- INT8 Model Size: 169.9 KB
- INT8 Test Accuracy: 90.92%
- Accuracy Drop: 0.24%
- Inference Time (INT8): 321.20 ms/batch

---

### Comparison: Static vs Dynamic Quantization

| Metric | Static INT8 | Dynamic INT8 |
|--------|-------------|--------------|
| Model Size | 176.0 KB | 169.9 KB |
| Accuracy | 90.91% | 90.92% |
| Accuracy Drop | 0.25% | 0.24% |
| Inference Time | 22.81 ms/batch | 321.20 ms/batch |

**Size:** Dynamic quantization produces a slightly smaller model (169.9 vs 176.0 KB) because it doesn't embed per-tensor activation quantization parameters (scales and zero-points) in the graph — those are computed on-the-fly.

**Accuracy:** Both methods achieve nearly identical accuracy (~0.25% drop), which is expected for a model with bounded activations (ReLU6, Sigmoid). In this case the pre-computed calibration ranges from static quantization offer no meaningful advantage over runtime-computed ranges.

**Inference time (the key difference):** Dynamic quantization is **14× slower** than static (321 ms vs 23 ms). This is because dynamic quantization must analyze each activation tensor at runtime to determine quantization ranges before performing INT8 arithmetic — effectively adding a profiling pass on every forward call. Static quantization bakes these ranges into the graph at export time, so inference executes pure INT8 kernels with zero overhead.

**Verdict:** For deployment, **static quantization is strictly preferred** — it delivers equivalent accuracy and size reduction while being an order of magnitude faster. Dynamic quantization's only advantage is convenience (no calibration data needed), making it useful for quick prototyping but not production inference.

# Summary Table

| Metric | FP32 (Original) | FP32 (ONNX) | INT8 Static | INT8 Dynamic (Optional) |
|--------|-----------------|-------------|-------------|--------------|
| Size (KB) | 318.3 | 325.2 | 176.0 | 169.9 |
| Accuracy (%) | 91.16 | 91.16 | 90.91 | 90.92 |
| Inference (ms) | 244.46 | 25.73 | 22.81 | 321.20 |